In [1]:
from api_neurotask import *
from baselines.metrics import get_R2
from bayes_opt import BayesianOptimization, UtilityFunction
from baselines.forecast import *
import pynapple as nap
import nemos as nmo

In [2]:
data = nap.load_file("001056/sub-Animal-1-&-2/sub-Animal-1-&-2.nwb")
df , bin = get_dataframe(data,filter_result=[b'R'])

Data loaded with bin size of 10.0 ms


In [3]:
new_bin_size = 20
df10 = rebin(df,prev_bin_size = bin ,new_bin_size = new_bin_size)

In [4]:
df10.head()

,session,trial_id,Neuron0,Neuron1,Neuron2,Neuron3,Neuron4,Neuron5,Neuron6,Neuron7,...,result,EventTarget_Onset,EventGo_cue,EventBump,target_dir,hand_vel_y,hand_vel_x,hand_pos_y,hand_pos_x,bump_dir
0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,b'R',False,False,False,0.0,13.844993,-9.663045,-40.768042,-2.777333,1.570796
1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,b'R',False,False,False,0.0,12.197587,-8.697542,-40.507926,-2.961398,1.570796
2,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,b'R',False,False,False,0.0,10.629333,-7.434564,-40.279349,-3.122960,1.570796
3,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,b'R',False,False,False,0.0,9.039913,-6.239253,-40.083160,-3.259433,1.570796
4,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,b'R',False,False,False,0.0,7.627443,-5.119604,-39.916148,-3.373000,1.570796


In [5]:
X_train_list, y_train_list, X_val_list, y_val_list, X_test_list, y_test_list= process_data_forecast_mimo(df10, bins_before=5,bins_after=1,
                                                 training_range=[0, 0.7], valid_range=[0.7, 0.8], testing_range=[0.8, 1], 
                                        filter_g=True, bin = new_bin_size, sigma= 50,scale = True)

In [7]:
print("There are", sum(X.shape[2] for X in X_train_list), "unique neurons in the dataset in",len(X_train_list),"different sessions")

There are 461 unique neurons in the dataset in 4 different sessions


In [8]:
def lstm_evaluate(num_units,frac_dropout,n_epochs):
            num_units=int(num_units)
            frac_dropout=float(frac_dropout)
            n_epochs=int(n_epochs)
            model_lstm=LSTMForecasting(units=num_units,dropout=frac_dropout,num_epochs=n_epochs,verbose=0, input_time_steps=5, output_time_steps=1)
            model_lstm.fit(X_train,y_train)
            y_valid_predicted_lstm=model_lstm.predict(X_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_lstm))

In [9]:
for i in range(len(X_train_list)):
        print('session ',i+1)
        X_train = X_train_list[i]
        X_valid = X_val_list[i]
        y_train = y_train_list[i]
        y_valid = y_val_list[i]
                
        #Do bayesian optimization
        lstmBO = BayesianOptimization(lstm_evaluate, {'num_units': (10, 500), 'frac_dropout': (0,.5), 'n_epochs': (2,25)})
        #lstmBO.maximize(init_points=20, n_iter=20, kappa=10)
        utility = UtilityFunction(kind="ucb", kappa=10, xi=0.0)
        lstmBO.maximize(init_points=1,n_iter=3)

        best_params=lstmBO.max['params']
        frac_dropout=float(best_params['frac_dropout'])
        n_epochs=int(best_params['n_epochs'])
        num_units=int(best_params['num_units'])

session  1
|   iter    |  target   | frac_d... | n_epochs  | num_units |
-------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
| 1         | 0.9749    | 0.1798    | 18.54     | 217.2     |


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
| 2         | 0.9654    | 0.04242   | 9.31      | 119.0     |


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
| 3         | 0.9749    | 0.1184    | 17.3      | 218.0     |


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


228/228 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
| 4         | 0.9805    | 0.01974   | 24.95     | 315.3     |
session  1
|   iter    |  target   | frac_d... | n_epochs  | num_units |
-------------------------------------------------------------


KeyboardInterrupt: 

In [30]:
#session id
i = 3

model_lstm=LSTMForecasting(units=218,dropout=0.1184,num_epochs=3,verbose=0, input_time_steps=5, output_time_steps=1)

#Fit model
model_lstm.fit(X_train_list[i],y_train_list[i])

#Get predictions
y_test_predicted_lstm=model_lstm.predict(X_test_list[i])

#Get metric of fit
R2s_lstm=get_R2(y_test_list[i],y_test_predicted_lstm)
print('session', i+1,'R2s:', R2s_lstm)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
session 4 R2s: [0.88739509]


In [27]:
_, _, _, _, _, y_test_true = process_data_forecast_mimo(df10, bins_before=5,bins_after=20,
                                                    training_range=[0, 0.7], valid_range=[0.7, 0.8], testing_range=[0.8, 1], 
                                            filter_g=True, bin = new_bin_size, sigma= 50,scale=True)

In [32]:
initial_input = X_test_list[i]
predictions = []
neurons = X_train_list[i].shape[2]
n_steps = 20

for _ in range(n_steps):
            prediction = model_lstm.predict(initial_input)
            prediction = np.clip(prediction, -2,2)
            predictions.append(prediction)
            # Update the input with the prediction for the next recursive step
            
            initial_input = np.concatenate((initial_input[:,:-1,:],prediction[:,-1:,:]),axis=1)

pred = np.concatenate(predictions, axis=1)

r2s = get_R2(y_test_true[i][:-n_steps,:,:],pred[:-n_steps,:,:])

print('session', i+1,'R2s:', r2s, '5: ', r2s[:5].mean(), '20: ', r2s.mean())

mse = (np.square(y_test_true[i][:-n_steps,:,:] -  pred[:-n_steps,:,:]).mean(axis=(0,2)))
print('session', i+1,'MSE:', mse, '5: ', mse[:5].mean(), '20: ', mse.mean())
mae = np.absolute((y_test_true[i][:-n_steps,:,:] -  pred[:-n_steps,:,:])).mean(axis=(0,2))
print('session', i+1,'MAE:', mae, '5: ', mae[:5].mean(), '20: ', mae.mean())

422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
session 4 R2s: [ 0.88730098  0.7504155   0.61401743  0.48400258  0.36196068  0.2466179
  0.13226132  0.00717265 -0.14670252 -0.35026297 -0.62079207 -0.96775652
 -1.38706611 -1.8665